In [14]:
import pandas as pd

# Cargar el dataset original
df = pd.read_csv('/content/traspdf_hasta_marzo.csv', sep=';', encoding='latin1', low_memory=False)

In [15]:
# Eliminar duplicados
df = df.drop_duplicates()

In [16]:
# Analizar valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())

Valores nulos por columna:
TREJERCI         0
TRFECINS         0
TRFE1MAT         0
TRFEPRIN    120994
TRVTIPVE         0
             ...  
TRD1002     307736
TRD1003     307655
TRD1004     307619
TRD1005     307502
TRD1006     306304
Length: 97, dtype: int64


In [17]:
# Codificar variables indicadores del tipo de fallo (TRDxxxx)

columnas_trd = [col for col in df.columns if col.startswith('TRD')]

# Codificación ordinal
def codificar_defecto(valor):
    if pd.isna(valor) or valor.strip() == '':
        return 0  # (caso en el que no hay defecto presente)
    elif 'LEVE' in valor:
        return 1
    elif 'GRAVE' in valor:
        return 2
    else:
        return 0


for col in columnas_trd:
    df[col] = df[col].apply(codificar_defecto)

In [18]:
# Tratamiento de variables temporales

# Convertir a formato datetime
df['TRFECINS'] = pd.to_datetime(df['TRFECINS'], errors='coerce')
df['TRFE1MAT'] = pd.to_datetime(df['TRFE1MAT'], errors='coerce')

# Ver cuántos valores nulos hay
nulos_fecins = df['TRFECINS'].isnull().sum()
nulos_fecmat = df['TRFE1MAT'].isnull().sum()

print(f'Nulos en TRFECINS: {nulos_fecins}')
print(f'Nulos en TRFE1MAT: {nulos_fecmat}')

# Si los nulos son muy pocos, eliminamos esas filas justificadamente
# Aquí usamos un umbral del 1%
umbral = 0.01 * len(df)

if nulos_fecins <= umbral:
    df = df.dropna(subset=['TRFECINS'])
else:
    fecha_media = df['TRFECINS'].dropna().median()
    df['TRFECINS'] = df['TRFECINS'].fillna(fecha_media)

if nulos_fecmat <= umbral:
    df = df.dropna(subset=['TRFE1MAT'])
else:
    fecha_media = df['TRFE1MAT'].dropna().median()
    df['TRFE1MAT'] = df['TRFE1MAT'].fillna(fecha_media)

# Calcular la antigüedad
df['antiguedad'] = (df['TRFECINS'] - df['TRFE1MAT']).dt.days // 365

# Si la antigüedad no se pudo calcular, imputamos con la mediana
df['antiguedad'] = df['antiguedad'].fillna(df['antiguedad'].median())

Nulos en TRFECINS: 0
Nulos en TRFE1MAT: 0


In [19]:
# Tratamiento de variables categóricas

df['TRVRESUL'] = df['TRVRESUL'].fillna('DESCONOCIDO')
df['TRVTIPVE'] = df['TRVTIPVE'].fillna('DESCONOCIDO')
df['TRVPROSE'] = df['TRVPROSE'].fillna('DESCONOCIDO')

# Función para codificar TRVRESUL
def codificar_resultado(valor):
    if isinstance(valor, str):
        valor = valor.upper()
        if 'FAVORABLE CON DEFECTOS LEVES' in valor:
            return 0
        elif 'DESFAVORABLE (ALGUN GRAVE)' in valor:
            return 1
        elif 'NEGATIVA (ALGUN MUY GRAVE)' in valor:
            return 2
    return 3  # Para desconocidos o valores raros

# Aplicar la codificación
df['target'] = df['TRVRESUL'].apply(codificar_resultado)

print("Distribución final de target:")
print(df['target'].value_counts())

# Variables dummies
df_encoded = pd.get_dummies(df[['TRVPROSE', 'TRVTIPVE']], drop_first=True)

Distribución final de target:
target
0    186810
1    117733
2      3255
Name: count, dtype: int64


In [20]:
# Creación de una variable de total de fallos por inspección

df['total_defectos'] = df[columnas_trd].apply(lambda fila: sum(fila > 0), axis=1)

In [21]:
# Combinar en el dataset final

final_df = pd.concat([
    df[['antiguedad', 'total_defectos', 'target']],
    df_encoded,
    df[columnas_trd]
], axis=1)

In [22]:
# Guardar y descargar nuevo dataset
final_df.to_csv('/content/dataset_vehiculos_limpio.csv', index=False)

from google.colab import files
files.download('/content/dataset_vehiculos_limpio.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>